In [ ]:
import cv2
import numpy as np

def puzzle(image1,image2,image3,image4):
    h1,w1,c1 = image1.shape
    h2,w2,c2 = image2.shape
    h3,w3,c3 = image3.shape
    h4,w4,c4 = image4.shape
    c=(c1+c2+c3+c4)
    if c!=4*c1 & c!=4*c2 & c!=4*c3 &c!=4*c4 :
        print("channels NOT match, cannot merge")
        return 
    else:
        h=1080
        Image1 = cv2.resize(image1, (h, h), interpolation=cv2.INTER_NEAREST)
        Image2 = cv2.resize(image2, (int(h/3), int(h/3)), interpolation=cv2.INTER_NEAREST)
        Image3 = cv2.resize(image3, (int(h/3), int(h/3)), interpolation=cv2.INTER_NEAREST)
        Image4 = cv2.resize(image4, (int(h/3), int(h/3)), interpolation=cv2.INTER_NEAREST)
        Image=np.hstack([Image2,Image3])
        Image=np.hstack([Image,Image4])
        Image=np.vstack([Image1,Image])
    return Image
    
def crop(path):
    im = cv2.imread(path)
    if im.shape[1]>=im.shape[0]:
        y=0
        Y=im.shape[0]
        x=(im.shape[1]-im.shape[0])/2
        X=im.shape[1]-(im.shape[1]-im.shape[0])/2
    else:
        x=0
        X=im.shape[1]
        y=(im.shape[0]-im.shape[1])/2
        Y=im.shape[0]-(im.shape[0]-im.shape[1])/2
    img=im[int(y):int(Y),int(x):int(X)]
    print(" x:",int(x)," X:",int(X),"y:",int(y),"Y:",int(Y))
    return img

path1="image1.png"
path2="image2.png"
path3="image3.png"
path4="image4.png"

image1=crop(path1)
image2=crop(path2)
image3=crop(path3)
image4=crop(path4)

print(image1.shape)

image=puzzle(image1,image2,image3,image4)
cv2.imwrite("image.png",image)

'''
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyWindow("Image") '''

In [1]:
import cv2
import numpy as np

def crop(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    
    low=cv2.medianBlur(gray,17) #中值滤波
    thresh= cv2.adaptiveThreshold(low,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,51,2)
    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))  # 矩形结构
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel3)

    erosion = cv2.erode(closed, kernel3)  # 腐蚀
    erosion = cv2.erode(erosion, kernel3)  # 腐蚀
    opened2 = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel3) #开运算
    closed2 = cv2.morphologyEx(opened2, cv2.MORPH_CLOSE, kernel3) #闭运算
    
    contours, hierarchy = cv2.findContours(closed2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    bestarea=0
    h=img.shape
    for i in range(0,len(contours)):
        x, y, w, h = cv2.boundingRect(contours[i])   
        area=w*h
        if area>bestarea :
            newimage=img[y+2:y+h-2,x+2:x+w-2] # 先用y确定高，再用x确定宽
            bestarea=area
    return newimage

def puzzle(image1,image2,image3,image4):
    h1,w1,c1 = image1.shape
    h2,w2,c2 = image2.shape
    h3,w3,c3 = image3.shape
    h4,w4,c4 = image4.shape
    
    c=(c1+c2+c3+c4)
    if c!=4*c1 & c!=4*c2 & c!=4*c3 &c!=4*c4 :
        print("channels NOT match, cannot merge")
        return 
    else:
        h=1080
        Image1 = cv2.resize(image1, (h, h), interpolation=cv2.INTER_NEAREST)
        Image2 = cv2.resize(image2, (int(h/3), int(h/3)), interpolation=cv2.INTER_NEAREST)
        Image3 = cv2.resize(image3, (int(h/3), int(h/3)), interpolation=cv2.INTER_NEAREST)
        Image4 = cv2.resize(image4, (int(h/3), int(h/3)), interpolation=cv2.INTER_NEAREST)
        Image=np.hstack([Image2,Image3])
        Image=np.hstack([Image,Image4])
        Image=np.vstack([Image1,Image])
    return Image

path1="image1.png"
path2="image2.png"
path3="image3.png"
path4="image4.png"

image1=crop(path1)
image2=crop(path2)
image3=crop(path3)
image4=crop(path4)

cv2.imshow("image1", image1)
cv2.imshow("image2", image2)
cv2.imshow("image3", image3)
cv2.imshow("image4", image4)
cv2.waitKey(0)
cv2.destroyAllWindows()

image=puzzle(image1,image2,image3,image4)
cv2.imwrite("image.png",image)

True